In [0]:
# Przydatne biblioteki to analizy danych
import pandas as pd
import numpy as np

# Ciekawe materiały w internecie

IPIPAN ma fajne demo, ale ostatnio często nie działa. Dostępne jest pod linkiem:
  http://multiservice.nlp.ipipan.waw.pl/en/
  
[Analogie w embeddingach](http://dsmodels.nlp.ipipan.waw.pl/analogy.html): słowo 1: "mężczyzna", słowo 2: "król", słowo 3: "kobieta" -> słowo 4: "królowa".

Współczesne dema można znaleźć tutaj: http://ws.clarin-pl.eu/
  
Słowosieć zawiera relacje pomiędzy słowami. Przykładowa strona dla słowa [jechać](http://plwordnet.pwr.wroc.pl/wordnet/77005502-2081-11e8-a0a3-37c4290c57bb).

Cała lista narzędzi: http://clip.ipipan.waw.pl/LRT 


Przykład tagera z ujednoznacznianiem: [http://ws.clarin-pl.eu/morphoDiTa.shtml](http://ws.clarin-pl.eu/morphoDiTa.shtml). Polecam sprawdzić na przykładzie:

`Ja mam dwa psy. Mamy w sumie dwie mamy. Mamy mają też dwie mamy.`

(warto wybrać zakładkę "Tekst" w wynikach i najeżdżać na słowa) i porównać wyniki z innym tagerem. Proponuję wkleić to też do dostępnego tam [parsera](http://ws.clarin-pl.eu/parser.shtml).


# Embeddingi (Zanurzenia)

Najpierw ściągnijmy zanurzenia dla j. polskiego. Uwaga: plik ma 900MB i będzie się ściągał wolno poza siecią wydziałową.

Poniższy plik na bazie modeli z [IPIPAN](http://dsmodels.nlp.ipipan.waw.pl/w2v.html). Pod tym linkiem można też zrobić to samo klikając.


In [0]:
data_url = 'https://www.mimuw.edu.pl/~mm319369/priv/d73890416bec03ff3e2b3756af8c941c/pl-embeddings-skip-noh.txt'

Jeśli chcecie zobaczyć, jak z tym zadziałać samodzielnie można go pobrać na różne sposoby.

Poniższą komórkę wykonajcie przy pracy z Colaboratory. System poprosi o dostęp do Waszego dysku na tym koncie – trzeba wejść w podany link, przeklikać się, skopiować podany kod i wcisnąć Enter.

Jeszcze niżej pozostałe opcje.

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#https://drive.google.com/file/d/1bbEW3xoO2lYGAdONfXwldmi6tAACLSWX/view?usp=sharing
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1bbEW3xoO2lYGAdONfXwldmi6tAACLSWX'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('pl-embeddings-skip-noh.txt')
print('Downloaded content'.format())

Poniższa wersja zadziała pod Linuksem i będzie wyświetlać pasek postępu.

In [0]:
!wget $data_url

**Poniższą komórkę uruchomcie, gdy inne nie działają.**

In [0]:
try:
    from urllib import urlretrieve
except:
    from urllib.request import urlretrieve
from tempfile import mktemp

name, hdrs = urlretrieve(data_url, 'pl-embeddings-skip-noh.txt')
print("Done")

## Praca z embeddingami
Załadujmy je najpierw do pamięci. Poniższe wyświetli wartości zanurzenia dla poszczególnym słów.

In [0]:
embeddings = pd.read_csv('pl-embeddings-skip-noh.txt', sep=' ', header=None)
embeddings.set_index(0, inplace=True)
embeddings.index.name = 'word'
del embeddings[101]
embeddings.head()

Poniższy kod normalizuje długość wektorów do 1, co jest przydatne przy liczeniu cosinusa kąta.

In [0]:
embeddings = embeddings.div(np.sqrt((embeddings**2).sum(axis=1)), axis=0)

Poniższa funkcja zwróci nam stopien podobienstwa między słowami.

`emedding.loc[x]` zwraca nam wektor reprezentujący słowo `x`



In [0]:
def similarity(v1, v2):
  return np.matmul(v1, v2.T)

In [0]:
#@title Słowa { run: "auto" }
word1 = "pies" #@param {type:"string"}
word2 = "kot" #@param {type:"string"}

embedding1 = embeddings.loc[word1]
embedding2 = embeddings.loc[word2]
similarity(embedding1, embedding2)

Poniższa funkcja pozwala nam znaleźć najbliższe `n` słów do danego wektora (po cosinusie kąta).

In [0]:
def get_closest(vect, n=5):
    return [embeddings.index[x] for x in np.argsort(np.matmul(vect, embeddings.T))[-n:][::-1]]

Sprawdźmy, co jest najbliższe słowu 'pies'.

In [0]:
#@title Słowo { run: "auto" }
word = "pies" #@param {type:"string"}
n = 5 #@param {type:"integer"}
embedding = embeddings.loc[word]
get_closest(embedding, n=n)

Analogie:
król - mężczyzna + kobieta = królowa

In [0]:
def get_analogy(base, minus, plus, n=5):
  return get_closest(embeddings.loc[base] - embeddings.loc[minus] + embeddings.loc[plus], n=n)

In [0]:
#@title Analogie { run: "auto" }
base = "kr\xF3l" #@param {type:"string"}
minus = "m\u0119\u017Cczyzna" #@param {type:"string"}
plus = "kobieta" #@param {type:"string"}
n = 5 #@param {type:"integer"}

get_analogy(base, minus, plus, n)

Co ciekawe, w j. polskim embeddingi kodują formę gramatyczną i działają na nich analogie:

In [0]:
get_analogy('samochodem', 'samochód', 'kot')

In [0]:
get_analogy('samochody', 'samochód', 'kot')

Zadanie dla rozrywki: napisz kod, który znajduje słowo niepasujące do pozostałych. Np. w `["kolacja", "obiad", "samochód", "śniadanie"]` niepasującym słowem jest `"samochód"`.

# NLTK

NLTK jest biblioteką w Pythonie do przetwarzania języka naturalnego. Ma bardzo wiele modułów, ale niestety wsparcie dla polskiego jest ograniczone.

Zaimportujmy oraz pobierzmy potrzebne moduły.

In [0]:
!pip install nltk

In [0]:
import nltk

In [0]:
nltk.download('popular')


In [0]:
#@title Sentence { run: "auto" }
sentence = "Employees of Cambridge Analytica, including the suspended CEO Alexander Nix, weren't filmed." #@param {type:"string"}


Tokenizacja: podział na słowa. (Zwróćcie uwagę na oddzielenie "n't".)

In [0]:
tokens = nltk.word_tokenize(sentence)
tokens

Tagowanie częścią mowy:

In [0]:
tags = nltk.pos_tag(tokens)
tags[:5]

Wykrywanie nazw własnych:

In [0]:
entities = nltk.chunk.ne_chunk(tags)
print(entities)

Poruszanie się po słowosieci:

In [0]:
from nltk.corpus import wordnet

Definicja:

In [0]:
syn = wordnet.synsets("NLP")
 
print(syn[0].definition())

Synonimy:

In [0]:
synonyms = []
 
for syn in wordnet.synsets('Computer'):
    for lemma in syn.lemmas():
        synonyms.append(lemma.name())
 
synonyms